## User Management
Interaction with EOEPCA endpoint using python Demo Client.

In [ ]:
import utils.DemoClient as client
import jwt
import json
import requests
from rauth import OAuth2Service

## Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [ ]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain= "demo.eoepca.org"
platform_domain= "auth." + base_domain
base_url= "https://" + platform_domain
dummy_serv= "https://dummy-service." + base_domain
dummy_pep_auth= "http://dummy-service-pep." + base_domain + "/authorize"
dummy_pep_api= "http://dummy-service-pepapi." + base_domain
pdp_url= "http://" + platform_domain + "/pdp"
authorize_url= 'https://' + platform_domain + '/oxauth/restv1/authorize'
token_url= 'https://' + platform_domain + '/oxauth/restv1/token'
redirect_uri_for_github= f"https://{platform_domain}/oxauth/auth/passport/img/github.png"
demo= client.DemoClient(base_url)
demo.register_client([redirect_uri_for_github])
demo.save_state()

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

### Authenticate

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as Eric (isOperator: True) and get ID Token
#-------------------------------------------------------------------------------
USER_NAME= "eric"
USER_PASSWORD= "defaultPWD"
operator_id_token= demo.get_id_token(USER_NAME, USER_PASSWORD)
print("operator_id_token:", operator_id_token)

client_id, client_secret= demo.get_client_credentials()

### Inspect the ID Token

In [ ]:
# Inspect ID Token
jwt_header= jwt.get_unverified_header(operator_id_token)
jwt_payload= jwt.decode(operator_id_token, options={"verify_signature": False})
print("JWT Header:", json.dumps(jwt_header, indent = 2), "\n---\nJWT Payload:", json.dumps(jwt_payload, indent = 2))
operator_UID= jwt_payload["sub"]

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## External ID Provider Login

The current External IdPs are GitHub and COIH, this test will apply the authorization code flow of OAuth2, where a first request to the authorization endpoint of the Login Service will return a code once the external user is authenticated. Afterwards the redirection will prompt a GitHub logo Image but in the url we can extract the code,that will be used to retrieve an access token. Copy the URL and proceed to next step

In [ ]:
#--------------------------
#LINK TO SOCIAL PASSPORT
#--------------------------

scope= ['openid', 'email', 'user_name', 'is_operator']
service= OAuth2Service(name="foo", client_id=client_id, client_secret=client_secret, access_token_url=token_url, authorize_url=authorize_url, base_url=base_url)
params= {'scope': scope,'response_type': 'code','redirect_uri': redirect_uri_for_github,'acr_values': 'passport_social'}

url= service.get_authorize_url(**params)
print("Link to Social Passport Login: (Login using GitHub option)")
print(url)

Here you will have to copy/paste the redirection once logged in to retrieve code and continue with the flow. An access token should result as output

In [ ]:
#--------------------------
#TOKEN RETRIEVAL
#--------------------------
code_url= input('Please paste redirect URL: ').strip()
code= code_url.split('code=')[1].split('&')[0]
data= {'client_id': client_id, 'client_secret': client_secret, 'code': code, 'grant_type': 'authorization_code', 'redirect_uri': redirect_uri_for_github}
response= requests.post(token_url, data=data)
print(response.text)

## Access Protected Endpoint
We perform an example access to an ADES resource.
The ADES is protected by a PEP (Policy Enforcement Point). When accessing the protected endpoint the client follows a UMA (User Managed Access) flow to obtain authorized access on behalf of the user. The UMA flow utilises the ID Token to obtain a 'ticket' which can then be exchanged by the client for an access token (RPT - Relying Party Token) that acts as a short-lived (e.g. 5 mins) credential for the specific access by a specific client, to a specific resource, on behalf of a specific user.
The use case will perform an update to the policy associated to the resource in order to allow access to a non operator user, external to the resource.

### Resource Insertion
The authenticated user must be an operator to realize this operation.
A call to the PEP resource API is done in order to include a new resource for the demo into the database. This resource has protected scope so only the operator user is able to access it.
If that resource already exists a 422 response will be returned.

In [ ]:
#--------------------------
#INSERT PROTECTED RESOURCE
#--------------------------
uri= "/demo/testResource"
data= '{"icon_uri":"'+uri+'","name":"DemoResource","scopes":["protected_access"]}'
headers= {'Content-Type': 'application/json, Authorization: Bearer '+ operator_id_token}
response= requests.post(dummy_pep_api+'/resources', headers=headers, data=data, verify=False)
print("Resource Insertion with code: " + str(response))

#--------------------------
#RESOURCE PUBLIC VISUALIZATION-->Query the resource endpoint to check if the resource is inserted
#--------------------------
operation_headers= { 'content-type': "application/x-www-form-urlencoded", "cache-control": "no-cache", "Authorization": "Bearer "+operator_id_token}
res= requests.get(dummy_pep_api +"/resources"+"?path="+uri, headers=headers, verify=False)
k= res.json()
print("Resource found in database: " + str(k))

### Operator successful access to resource
The operator will perform the UMA Flow to prove a successful access to the resource created.

In [ ]:
#--------------------------
#GET TICKET-->Naive request to resource with no authorization, ticket returned
#--------------------------
rpt= None
print("ADES API Processes endpoint:", dummy_pep_auth)
ticket_headers_oper= { 'content-type': "application/json", "cache-control": "no-cache", "Authorization": "Bearer " + operator_id_token, "X-Original-Uri": uri }
tkt= requests.get(dummy_pep_auth, headers=ticket_headers_oper, verify=False)
ticket= tkt.headers["WWW-Authenticate"].split("ticket=")[1]
print("401 Unauthorized, ticket returned: " + ticket)

#--------------------------
#GET RPT-->Request RPT token from UMA endpoints giving a ticket and the user id_token
#--------------------------
data= "claim_token_format=http://openid.net/specs/openid-connect-core-1_0.html#IDToken&claim_token="+operator_id_token+"&ticket="+ticket+"&grant_type=urn%3Aietf%3Aparams%3Aoauth%3Agrant-type%3Auma-ticket&client_id="+client_id+"&client_secret="+client_secret+"&scope=openid"
rpt_headers= { 'Content-Type': 'application/x-www-form-urlencoded', 'cache-control': 'no-cache' }
rpts_response= requests.post(token_url, data=data, headers=rpt_headers, verify = False)
rpt= rpts_response.json()["access_token"]
print("The UMA returns: " + str(rpts_response) + " with a RPT UMA Token: " + str(rpt))

#--------------------------
#ACCESS TO RESOURCE-->The access_token is returned and the request to the resource is retried with approved credentials
#--------------------------
headers= { 'content-type': "application/json", "cache-control": "no-cache", "Authorization": "Bearer "+rpt, "X-Original-Uri": uri}
resource_access= requests.get(dummy_pep_auth, headers=headers, verify=False)
print("Retrying access to resource with proper token: " + str(resource_access))

### Non Operator denied access to resource
The Alice user (not an operator) will perform the UMA Flow to prove an unsuccessful attempt to resource access.
The Login Service will call the PDP to check if the policy associated to the resource allows access to Alice. Once the PDP checks the resource is not accessible by Alice, the RPT request will respond with a 403 Forbidden by Policy

In [ ]:
#--------------------------
#USER AUTH
#--------------------------
USER_NAME= "alice"
USER_PASSWORD= "defaultPWD"
alice_id_token= demo.get_id_token(USER_NAME, USER_PASSWORD)
alice_UID= jwt.decode(alice_id_token, options={"verify_signature": False})["sub"]

#--------------------------
#GET TICKET--> 401
#--------------------------
ticket_headers_alice= { 'content-type': "application/json", "cache-control": "no-cache", "Authorization": "Bearer "+alice_id_token, "X-Original-Uri": uri}
tkt= requests.get(dummy_pep_auth, headers=ticket_headers_alice, verify=False)
ticket= tkt.headers["WWW-Authenticate"].split("ticket=")[1]
print("401 Unauthorized, ticket returned: " + str(ticket))
#--------------------------
#GET RPT--> 403
#--------------------------
data= "claim_token_format=http://openid.net/specs/openid-connect-core-1_0.html#IDToken&claim_token="+alice_id_token+"&ticket="+ticket+"&grant_type=urn%3Aietf%3Aparams%3Aoauth%3Agrant-type%3Auma-ticket&client_id="+client_id+"&client_secret="+client_secret+"&scope=openid"
rpts_response= requests.post(token_url, data=data, headers=rpt_headers, verify = False)
print("Expected to be forbidden for non operator user (403 code): " + str(rpts_response))

### Update Policy
The operator user who is the resource owner will perform a call to the PDP to update the policy associated to that resource to include access to Alice

In [ ]:
#--------------------------
#UPDATE POLICY--> 200
#--------------------------
data= {'name':'Updated Policy for Demo Resource','description':'Policy modified for demo purpose','config':{'resource_id':k["_id"] ,'action':'get','rules':[{'OR':[{'EQUAL':{'id':operator_UID}},{'EQUAL':{'id':alice_UID}}]}]},'scopes':['protected_access']}
resp, text= demo.update_policy(pdp_url, data, k["_id"], operator_id_token)
print(resp)

### Non Operator successful access to resource
With the new policy the user Alice will perform the same request to the resource but now with granted access.

In [ ]:
#--------------------------
#GET TICKET--> 401
#--------------------------
tkt= requests.get(dummy_pep_auth, headers=ticket_headers_alice, verify=False)
ticket= tkt.headers["WWW-Authenticate"].split("ticket=")[1]
print("401 Unauthorized, ticket returned: " + ticket)
#--------------------------
#GET RPT--> 200
#--------------------------
data= "claim_token_format=http://openid.net/specs/openid-connect-core-1_0.html#IDToken&claim_token="+alice_id_token+"&ticket="+ticket+"&grant_type=urn%3Aietf%3Aparams%3Aoauth%3Agrant-type%3Auma-ticket&client_id="+client_id+"&client_secret="+client_secret+"&scope=openid"
rpts_response= requests.post(token_url, data=data, headers=rpt_headers, verify = False)
rpt= rpts_response.json()["access_token"]
print("The UMA returns: " + str(rpts_response) + " with a RPT UMA Token" + str(rpt))
#--------------------------
#ACCESS TO RESOURCE--> 200
#--------------------------
headers= { 'content-type': "application/json", "cache-control": "no-cache", "Authorization": "Bearer "+rpt, "X-Original-Uri": uri}
tkt= requests.get(dummy_pep_auth, headers=headers, verify=False)
print("Retrying access to resource with proper token: " + str(tkt))

### Open Resource Insertion
The operator will insert a new resource with `open` scope. This will allow to any non authenticated person to access the resource, which means that no authorization header needs to be added in the request. The registration of open resources will still be limited only to operators to avoid unwanted insertions.
After the insertion the resources endpoint will be called to list all resources accesible by the operator, which will list the protected and the open resource used in the current Notebook

In [ ]:
#--------------------------
#INSERT OPEN RESOURCE
#--------------------------
open_uri= uri + "/openException"
data= '{"icon_uri":"'+open_uri+'","name":"OpenResource","resource_scopes":["open"]}'
headers= {'Content-Type': 'application/json, Authorization: Bearer '+ operator_id_token}
response= requests.post(dummy_pep_api+'/resources', headers=headers, data=data, verify=False)
print("Resource Insertion with code: " + str(response))

#--------------------------
#RESOURCE OPEN VISUALIZATION-->Query the resource endpoint to check if the open resource is inserted as an operator
#--------------------------
operation_headers= { 'content-type': "application/x-www-form-urlencoded", "cache-control": "no-cache", "Authorization": "Bearer "+operator_id_token}
res= requests.get(dummy_pep_api +"/resources", headers=headers, verify=False)
k= res.json()
print("All resources accesible by the user found in database: ")
print(str(k))

### Non-Authenticated Access to resources
In this case, a person outside the identity platform will try to access the two resources. The person will first list all open resources by interacting with the PEP resource API and then will check the protection in both cases by accessing the resources. First an attempt to the protected one, which will imply an Unauthorized response from the PEP. Finally a call to the open resource that will result in a direct redirection to the resource and a 200 status code. Note that the user wont use this time any Authorization header on the request to the services.

In [ ]:
#--------------------------
#RESOURCE OPEN VISUALIZATION-->Query the resource endpoint to check if the open resource is visible by non authenticated person (no token required)
#--------------------------
res= requests.get(dummy_pep_api +"/resources", verify=False)
k= res.json()
print("Open resource found in database: ")
print(str(k))

#--------------------------
#PROTECTED ACCESS TO RESOURCE BY NON AUTHENTICATED PERSON-->Query the protected resource to check if the non authenticated person has access (401 expected)
#--------------------------
a= requests.get(dummy_serv + uri, verify=False)
print("Unauthorize access to protected resource (no token passed), expected 401: "+ str(a))

#--------------------------
#OPEN ACCESS TO RESOURCE BY NON AUTHENTICATED PERSON-->Query the protected resource to check if the non authenticated person has access (200 expected)
#--------------------------
a= requests.get(dummy_serv + open_uri, verify=False)
print("Unauthorize access to open resource (no token passed), expected 200: "+ str(a))

### Terms and conditions

In [ ]:
USER_NAME= "eric"
USER_PASSWORD= "defaultPWD"
operator_id_token= demo.get_id_token(USER_NAME, USER_PASSWORD)
uri = "/demo-terms-conditions"
print("dummy_pep_api " + (dummy_pep_api+'/resources'))
data = '{"icon_uri":"'+uri+'","name":"TermsConditionsResource","scopes":["protected_access"],"T&C":{"term_id": "This is a term id","term_description": "This is a term description"}}'
headers = {'Content-Type': 'application/json, Authorization: Bearer '+ operator_id_token}
response = requests.post(dummy_pep_api+'/resources', headers=headers, data=data, verify=False)
print("Resource Insertion with code (200 is ok, 422 is resource already inserted): " + str(response))